In [1]:
# Some imports
import sys
import os, glob
from mfx.processlocalizations import ProcessLocalizations
import re
from matplotlib import pyplot as plt
import math
import numpy as np
from mfx import mfxcolnames as col

In [2]:
# Get file names

OUTDIR_LOC = 'C:/Users/apoliti/Desktop/mflux_zarr_tmp_storage/analysis/' # Main directory to store zarr files
OUTDIR_REM =  'Z:/siva_minflux/analysis/'  # Main directory to store results 
INDIR  = 'Z:/siva_minflux/data/'       # main directory of msr file

# Multiple washes with different imager strand
indir_mwash = {'Syp_ATG9': INDIR + '/Multiwash/Syp_ATG9/', 
             'ZnT3_Syp': INDIR + '/Multiwash/ZnT3_Syp/'}
outdir_mwash = {'Syp_ATG9': OUTDIR_REM +  '/Multiwash/Syp_ATG9/', 
                'ZnT3_Syp': OUTDIR_REM +  '/Multiwash/ZnT3_Syp/'}
zarrdir_mwash = {'Syp_ATG9': OUTDIR_LOC + '/Multiwash/Syp_ATG9', 
                'ZnT3_Syp': OUTDIR_LOC + '/Multiwash/ZnT3_Syp/'}

# Wash with a single imager strand
indir_swash = {'Syp': INDIR + '/Single wash/Syp/', 
             'ATG9': INDIR + '/Single wash/ATG9/'}
outdir_swash = {'Syp': OUTDIR_REM +  '/Single wash/Syp/', 
                'ATG9': OUTDIR_REM +  '/Single wash/ATG9/'}
zarrdir_swash = {'Syp': OUTDIR_LOC +  '/Single wash/Syp/', 
                'ATG9': OUTDIR_LOC +  '/Single wash/ATG9/'}


# Consitency controls. Wash with a single imager strand but multiple times. 
indir_cwash = {'VGLUT1_VGLUT1': INDIR + '/Multiwash/VGLUT1_VGLUT1/'}
outdir_cwash = {'VGLUT1_VGLUT1': OUTDIR_REM +  '/Multiwash/VGLUT1_VGLUT1/'}
zarrdir_cwash = {'VGLUT1_VGLUT1': OUTDIR_LOC +  '/Multiwash/VGLUT1_VGLUT1/'}



npy_mwash = {'Syp_ATG9': [], 'ZnT3_Syp': []}

for key in outdir_mwash:
    for (root, dirs, files) in os.walk(outdir_mwash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_mwash[key].append(npyfile)
                    

npy_swash = {'Syp': [], 'ATG9': []}

for key in outdir_swash:
    for (root, dirs, files) in os.walk(outdir_swash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_swash[key].append(npyfile)

npy_cwash = {'VGLUT1_VGLUT1': []}

for key in outdir_cwash:
    for (root, dirs, files) in os.walk(outdir_cwash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_cwash[key].append(npyfile)

pl = ProcessLocalizations(npy_cwash['VGLUT1_VGLUT1'][1])
pl.log_parameters()

2022-09-22 10:30:00,984 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220811_VGLUT1_ROI01_First\220811_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-08
DBCLUSTER_EPS_ALL 2e-08
DBCLUSTER_EPS_MERGED_ALL 2e-08
DBCLUSTER_EPS_MERGED_MEAS 2e-08
ref_error {'se_xy': 1.731907374501148e-09, 'se_z': 1.5518753909999794e-09, 'std_xy': 3.463814749002296e-09, 'std_z': 3.103750781999959e-09}
logger <Logger processlocalizations (INFO)>



In [13]:
def processFile(file_name, eps_range):
    pl = ProcessLocalizations(file_name)
    pl.trim_min_localizations()
    # Split tracks if needed, remove outliers in each track
    pl.cluster_tid(method=pl.CLS_METHOD_BAYES_GMM)

    # find objects that belong to the same cluster according to distance 30 nm
    for eps in eps_range:
            
        pl.DBCLUSTER_EPS_MEAS = eps
        pl.DBCLUSTER_EPS_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_MEAS = eps
        pl.log_parameters()
    
        pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all_intersect(col.CLS_ALL)
        summary_dict = pl.summary_per_tid2()
        pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all_intersect(col.CLS_ALL)
        summary_dict = pl.summary_per_tid2()
        eps_txt = '%.0f' % (eps*1e9)
        pl.export_csv(summary_dict, file_path=pl.file_path_no_ext() + '_eps' + eps_txt)
        pl.export_vtu(in_dict=summary_dict, coord=col.LTR, file_path=pl.file_path_no_ext() + '_eps' +  eps_txt)
    

In [14]:
eps_range = [2e-9, 5e-9, 1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 1e-7]
for label in npy_swash:
    for afile in npy_swash[label]:
        processFile(afile, eps_range=eps_range)

2022-09-22 11:19:00,083 [INFO] **** trim_min_localizations ****
220825_Syp_P1 Removed 281/1354 tracks with less than 3 localizations

2022-09-22 11:19:11,620 [INFO] **** cluster_tid ****
220825_Syp_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.03 nm
Processed TID: 228 / 1073, Total tracks TID2: 1205

2022-09-22 11:19:11,621 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/Syp/220825_Syp_ROI2\220825_Syp_ROI2.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-09
DBCLUSTER_EPS_ALL 2e-09
DBCLUSTER_EPS_MERGED_ALL 2e-09
DBCLUSTER_EPS_MERGED_MEAS 2e-09
ref_error {'se_xy': 1.9457500536181486e-09, 'se_z': 2.0569079892663198e-09, 'std_xy': 3.891500107236297e-09, 'std_z': 4.1138159785326395e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:19:11,805 [INFO] **** cluster_meas ****
220825_Syp_P1, Method dbscan, eps: 2.00 nm
Total tracks TID2: 1204, total clusters meas: 1204

2022-09-22 11:19:11

2022-09-22 11:19:20,318 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:19:21,092 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/Syp/220825_Syp_ROI2\220825_Syp_ROI2.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 5e-08
DBCLUSTER_EPS_ALL 5e-08
DBCLUSTER_EPS_MERGED_ALL 5e-08
DBCLUSTER_EPS_MERGED_MEAS 5e-08
ref_error {'se_xy': 1.9457500536181486e-09, 'se_z': 2.0569079892663198e-09, 'std_xy': 3.891500107236297e-09, 'std_z': 4.1138159785326395e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:19:21,439 [INFO] **** cluster_meas ****
220825_Syp_P1, Method dbscan, eps: 50.00 nm
Total tracks TID2: 1204, total clusters meas: 84

2022-09-22 11:19:21,441 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:19:21,443 [INFO] **** cluster_all_intersect ****
Only one wash
2022-09-22 11:19:22

2022-09-22 11:19:31,319 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:19:31,322 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:19:31,660 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/Syp/220825_Syp_ROI3\220825_Syp_ROI3.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 3e-08
DBCLUSTER_EPS_ALL 3e-08
DBCLUSTER_EPS_MERGED_ALL 3e-08
DBCLUSTER_EPS_MERGED_MEAS 3e-08
ref_error {'se_xy': 2.3814054942194073e-09, 'se_z': 2.213309655021242e-09, 'std_xy': 4.124715309411686e-09, 'std_z': 3.833564775379535e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:19:31,791 [INFO] **** cluster_meas ****
220825_Syp_P1, Method dbscan, eps: 30.00 nm
Total tracks TID2: 616, total clusters meas: 155

2022-09-22 11:19:31,793 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Noth

2022-09-22 11:19:38,807 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:19:38,809 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:19:39,081 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/Syp/220825_Syp_ROI4\220825_Syp_ROI4.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 1e-08
DBCLUSTER_EPS_ALL 1e-08
DBCLUSTER_EPS_MERGED_ALL 1e-08
DBCLUSTER_EPS_MERGED_MEAS 1e-08
ref_error {'se_xy': 2.1022603990135522e-09, 'se_z': 2.279814191212894e-09, 'std_xy': 3.641221821831493e-09, 'std_z': 3.94875401099728e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:19:39,155 [INFO] **** cluster_meas ****
220825_Syp_P1, Method dbscan, eps: 10.00 nm
Total tracks TID2: 500, total clusters meas: 342

2022-09-22 11:19:39,157 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothi

export csv table
export paraview file
2022-09-22 11:19:42,808 [INFO] **** trim_min_localizations ****
220825_Syp-P1 Removed 157/724 tracks with less than 3 localizations

2022-09-22 11:19:47,337 [INFO] **** cluster_tid ****
220825_Syp-P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.11 nm
Processed TID: 126 / 567, Total tracks TID2: 661

2022-09-22 11:19:47,339 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/Syp/220825_Syp_ROI5\220825_Syp_ROI5.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-09
DBCLUSTER_EPS_ALL 2e-09
DBCLUSTER_EPS_MERGED_ALL 2e-09
DBCLUSTER_EPS_MERGED_MEAS 2e-09
ref_error {'se_xy': 2.288078096368905e-09, 'se_z': 2.1124363629715803e-09, 'std_xy': 5.116298161309187e-09, 'std_z': 4.723551305746874e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:19:47,439 [INFO] **** cluster_meas ****
220825_Syp-P1, Method dbscan, eps: 2.00 nm
Total tracks TID2: 660, total clusters

...

2022-09-22 11:21:01,895 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:21:02,275 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/ATG9/220825_ATG9_ROI3\220825_ATG9_ROI3.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 5e-08
DBCLUSTER_EPS_ALL 5e-08
DBCLUSTER_EPS_MERGED_ALL 5e-08
DBCLUSTER_EPS_MERGED_MEAS 5e-08
ref_error {'se_xy': 2.5914422595097797e-09, 'se_z': 2.5313423304039118e-09, 'std_xy': 4.48850965835203e-09, 'std_z': 4.384413527609379e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:21:02,436 [INFO] **** cluster_meas ****
220825_ATG9_P2, Method dbscan, eps: 50.00 nm
Total tracks TID2: 626, total clusters meas: 86

2022-09-22 11:21:02,437 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:21:02,438 [INFO] **** cluster_all_intersect ****
Only one wash
2022-09-22 11:21:0

2022-09-22 11:21:08,572 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:21:08,573 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:21:08,854 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/ATG9/220825_ATG9_ROI4\220825_ATG9_ROI4.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 3e-08
DBCLUSTER_EPS_ALL 3e-08
DBCLUSTER_EPS_MERGED_ALL 3e-08
DBCLUSTER_EPS_MERGED_MEAS 3e-08
ref_error {'se_xy': 2.4262606713315665e-09, 'se_z': 2.5690758347695274e-09, 'std_xy': 4.202406755152446e-09, 'std_z': 4.449769874318247e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:21:08,946 [INFO] **** cluster_meas ****
220825_ATG9_P2, Method dbscan, eps: 30.00 nm
Total tracks TID2: 487, total clusters meas: 106

2022-09-22 11:21:08,948 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash.

2022-09-22 11:21:15,579 [INFO] **** cluster_meas ****
220825_ATG9_P2, Method dbscan, eps: 5.00 nm
Total tracks TID2: 545, total clusters meas: 468

2022-09-22 11:21:15,581 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:21:15,583 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:21:15,892 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/ATG9/220825_ATG9_ROI5\220825_ATG9_ROI5.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 1e-08
DBCLUSTER_EPS_ALL 1e-08
DBCLUSTER_EPS_MERGED_ALL 1e-08
DBCLUSTER_EPS_MERGED_MEAS 1e-08
ref_error {'se_xy': 3.1736576244831263e-09, 'se_z': 2.5657256936050177e-09, 'std_xy': 4.4882296548728166e-09, 'std_z': 3.6284840732253323e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:21:15,989 [INFO] **** cluster_meas ****
220825_ATG9_P2, Method dbscan, eps: 10

2022-09-22 11:21:19,945 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:21:19,946 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:21:20,487 [INFO] **** trim_min_localizations ****
220825_ATG9_P2 Removed 237/1208 tracks with less than 3 localizations

2022-09-22 11:21:31,995 [INFO] **** cluster_tid ****
220825_ATG9_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.65 nm
Processed TID: 216 / 971, Total tracks TID2: 1152

2022-09-22 11:21:31,997 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/ATG9/220825_ATG9_ROI6\220825_ATG9_ROI6.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-09
DBCLUSTER_EPS_ALL 2e-09
DBCLUSTER_EPS_MERGED_ALL 2e-09
DBCLUSTER_EPS_MERGED_MEAS 2e-09
ref_error {'se_xy': 3.297471097368615e-09, 'se_z': 2.9036459199066262e-09, 'std_xy': 5.7113874771323414e-09, 'std_z': 5.02926226

2022-09-22 11:21:42,547 [INFO] **** cluster_meas ****
220825_ATG9_P2, Method dbscan, eps: 40.00 nm
Total tracks TID2: 1151, total clusters meas: 124

2022-09-22 11:21:42,548 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-22 11:21:42,550 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-22 11:21:43,322 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/ATG9/220825_ATG9_ROI6\220825_ATG9_ROI6.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 5e-08
DBCLUSTER_EPS_ALL 5e-08
DBCLUSTER_EPS_MERGED_ALL 5e-08
DBCLUSTER_EPS_MERGED_MEAS 5e-08
ref_error {'se_xy': 3.297471097368615e-09, 'se_z': 2.9036459199066262e-09, 'std_xy': 5.7113874771323414e-09, 'std_z': 5.029262260468347e-09}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:21:43,836 [INFO] **** cluster_meas ****
220825_ATG9_P2, Method dbscan, eps: 50

In [15]:
eps_range = [2e-9, 5e-9, 1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 1e-7]
for label in npy_mwash:
    for afile in npy_mwash[label]:
        processFile(afile, eps_range=eps_range)

2022-09-22 11:21:48,056 [INFO] **** trim_min_localizations ****
220510-Syp_P1 Removed 132/615 tracks with less than 3 localizations
220510-ATG9_P2 Removed 50/392 tracks with less than 3 localizations

2022-09-22 11:21:57,230 [INFO] **** cluster_tid ****
220510-Syp_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.00 nm
Processed TID: 124 / 483, Total tracks TID2: 583
220510-ATG9_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.00 nm
Processed TID: 62 / 342, Total tracks TID2: 400

2022-09-22 11:21:57,233 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/Syp_ATG9/220510_Syp_ATG9_ROI01\220510_Syp_ATG9_ROI01.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-09
DBCLUSTER_EPS_ALL 2e-09
DBCLUSTER_EPS_MERGED_ALL 2e-09
DBCLUSTER_EPS_MERGED_MEAS 2e-09
ref_error {'se_xy': 1.5907650283037244e-09, 'se_z': 7.277297875402847e-10, 'std_xy': 3.896562620008164e-09, 'std_z': 1.7825666500977086e-09}
logger <Logger processloc

2022-09-22 11:22:10,822 [INFO] **** cluster_all ****
220510-Syp_P1, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 582, total cluster all: 133
220510-ATG9_P2, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 399, total cluster all: 52

2022-09-22 11:22:10,917 [INFO] **** cluster_all_intersect ****
220510-Syp_P1 cluster with 220510-ATG9_P2 cls 8/132 = 0.06TID2 354/582 = 0.61
220510-ATG9_P2 cluster with 220510-Syp_P1 cls 8/52 = 0.15TID2 313/399 = 0.78

export csv table
export paraview file
2022-09-22 11:22:11,699 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/Syp_ATG9/220510_Syp_ATG9_ROI01\220510_Syp_ATG9_ROI01.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 3e-08
DBCLUSTER_EPS_ALL 3e-08
DBCLUSTER_EPS_MERGED_ALL 3e-08
DBCLUSTER_EPS_MERGED_MEAS 3e-08
ref_error {'se_xy': 1.5907650283037244e-09, 'se_z': 7.277297875402847e-10, 'std_xy': 3.896562620008164e-09, 'std_z': 1.7825666500977086e-09}
log

2022-09-22 11:22:46,242 [INFO] **** cluster_all ****
220510-Syp_P1, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 582, total cluster all: 62
220510-ATG9_P2, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 399, total cluster all: 27

2022-09-22 11:22:46,328 [INFO] **** cluster_all_intersect ****
220510-Syp_P1 cluster with 220510-ATG9_P2 cls 10/61 = 0.16TID2 388/582 = 0.67
220510-ATG9_P2 cluster with 220510-Syp_P1 cls 10/26 = 0.38TID2 351/399 = 0.88

export csv table
export paraview file
2022-09-22 11:22:47,165 [INFO] **** trim_min_localizations ****
220510_Syp_P1 Removed 251/1295 tracks with less than 3 localizations
220510_ATG9_P2 Removed 114/948 tracks with less than 3 localizations

2022-09-22 11:23:24,111 [INFO] **** cluster_tid ****
220510_Syp_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 9.98 nm
Processed TID: 260 / 1044, Total tracks TID2: 1265
220510_ATG9_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 9.98 nm
Processed TID: 171 / 834, Total tracks TID2: 999

2022-09-22 11:23:24,113 [IN

2022-09-22 11:23:49,087 [INFO] **** cluster_all_intersect ****
220510_Syp_P1 cluster with 220510_ATG9_P2 cls 38/158 = 0.24TID2 786/1264 = 0.62
220510_ATG9_P2 cluster with 220510_Syp_P1 cls 38/100 = 0.38TID2 697/998 = 0.70

2022-09-22 11:23:52,125 [INFO] **** cluster_meas ****
220510_Syp_P1, Method dbscan, eps: 20.00 nm
Total tracks TID2: 1264, total clusters meas: 170
220510_ATG9_P2, Method dbscan, eps: 20.00 nm
Total tracks TID2: 998, total clusters meas: 123

2022-09-22 11:23:54,144 [INFO] **** cluster_all ****
220510_Syp_P1, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 1264, total cluster all: 159
220510_ATG9_P2, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 998, total cluster all: 101

2022-09-22 11:23:54,312 [INFO] **** cluster_all_intersect ****
220510_Syp_P1 cluster with 220510_ATG9_P2 cls 38/158 = 0.24TID2 786/1264 = 0.62
220510_ATG9_P2 cluster with 220510_Syp_P1 cls 38/100 = 0.38TID2 697/998 = 0.70

export csv table
export paraview file
2022-09-22 11:23:56,146 [INFO] **

2022-09-22 11:24:57,049 [INFO] **** cluster_all_intersect ****
220510_Syp_P1 cluster with 220510_ATG9_P2 cls 6/33 = 0.18TID2 76/1264 = 0.06
220510_ATG9_P2 cluster with 220510_Syp_P1 cls 6/17 = 0.35TID2 9/998 = 0.01

2022-09-22 11:25:02,333 [INFO] **** cluster_meas ****
220510_Syp_P1, Method dbscan, eps: 100.00 nm
Total tracks TID2: 1264, total clusters meas: 48
220510_ATG9_P2, Method dbscan, eps: 100.00 nm
Total tracks TID2: 998, total clusters meas: 23

2022-09-22 11:25:08,143 [INFO] **** cluster_all ****
220510_Syp_P1, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 1264, total cluster all: 34
220510_ATG9_P2, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 998, total cluster all: 18

2022-09-22 11:25:08,315 [INFO] **** cluster_all_intersect ****
220510_Syp_P1 cluster with 220510_ATG9_P2 cls 6/33 = 0.18TID2 76/1264 = 0.06
220510_ATG9_P2 cluster with 220510_Syp_P1 cls 6/17 = 0.35TID2 9/998 = 0.01

export csv table
export paraview file
2022-09-22 11:25:10,293 [INFO] **** trim_min_lo

...

2022-09-22 11:29:03,617 [INFO] **** cluster_all ****
220309_ZnT3_P1, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 113, total cluster all: 31
220309_Syp_P2, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 1132, total cluster all: 77

2022-09-22 11:29:03,692 [INFO] **** cluster_all_intersect ****
220309_ZnT3_P1 cluster with 220309_Syp_P2 cls 18/30 = 0.60TID2 94/113 = 0.83
220309_Syp_P2 cluster with 220309_ZnT3_P1 cls 18/77 = 0.23TID2 918/1132 = 0.81

export csv table
export paraview file
2022-09-22 11:29:04,622 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/ZnT3_Syp/220309_ZnT3_Syp_ROI01\220309_ZnT3_Syp_ROI01.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 5e-08
DBCLUSTER_EPS_ALL 5e-08
DBCLUSTER_EPS_MERGED_ALL 5e-08
DBCLUSTER_EPS_MERGED_MEAS 5e-08
ref_error {'se_xy': 8.706654263927838e-09, 'se_z': 5.531878064392318e-09, 'std_xy': 1.508036754905922e-08, 'std_z': 9.58149386880327e-09}
logge

2022-09-22 11:29:24,444 [INFO] **** cluster_all_intersect ****
220309_ZnT3_P1 cluster with 220309_Syp_P2 cls 20/190 = 0.11TID2 79/318 = 0.25
220309_Syp_P2 cluster with 220309_ZnT3_P1 cls 20/330 = 0.06TID2 199/580 = 0.34

2022-09-22 11:29:24,873 [INFO] **** cluster_meas ****
220309_ZnT3_P1, Method dbscan, eps: 5.00 nm
Total tracks TID2: 318, total clusters meas: 210
220309_Syp_P2, Method dbscan, eps: 5.00 nm
Total tracks TID2: 580, total clusters meas: 363

2022-09-22 11:29:25,118 [INFO] **** cluster_all ****
220309_ZnT3_P1, Method: dbscan, eps: 5.00 nm
Total tracks TID2: 318, total cluster all: 191
220309_Syp_P2, Method: dbscan, eps: 5.00 nm
Total tracks TID2: 580, total cluster all: 331

2022-09-22 11:29:25,184 [INFO] **** cluster_all_intersect ****
220309_ZnT3_P1 cluster with 220309_Syp_P2 cls 20/190 = 0.11TID2 79/318 = 0.25
220309_Syp_P2 cluster with 220309_ZnT3_P1 cls 20/330 = 0.06TID2 199/580 = 0.34

export csv table
export paraview file
2022-09-22 11:29:25,738 [INFO] **** log_par

2022-09-22 11:29:34,045 [INFO] **** cluster_all_intersect ****
220309_ZnT3_P1 cluster with 220309_Syp_P2 cls 36/81 = 0.44TID2 213/318 = 0.67
220309_Syp_P2 cluster with 220309_ZnT3_P1 cls 36/107 = 0.34TID2 446/580 = 0.77

2022-09-22 11:29:34,646 [INFO] **** cluster_meas ****
220309_ZnT3_P1, Method dbscan, eps: 40.00 nm
Total tracks TID2: 318, total clusters meas: 87
220309_Syp_P2, Method dbscan, eps: 40.00 nm
Total tracks TID2: 580, total clusters meas: 112

2022-09-22 11:29:35,171 [INFO] **** cluster_all ****
220309_ZnT3_P1, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 318, total cluster all: 82
220309_Syp_P2, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 580, total cluster all: 108

2022-09-22 11:29:35,229 [INFO] **** cluster_all_intersect ****
220309_ZnT3_P1 cluster with 220309_Syp_P2 cls 36/81 = 0.44TID2 213/318 = 0.67
220309_Syp_P2 cluster with 220309_ZnT3_P1 cls 36/107 = 0.34TID2 446/580 = 0.77

export csv table
export paraview file
2022-09-22 11:29:35,767 [INFO] **** log_p

2022-09-22 11:32:24,804 [INFO] **** cluster_meas ****
220309_ZnT3_P1, Method dbscan, eps: 5.00 nm
Total tracks TID2: 2678, total clusters meas: 2299
220309_Syp_P2, Method dbscan, eps: 5.00 nm
Total tracks TID2: 2401, total clusters meas: 2106

2022-09-22 11:32:26,091 [INFO] **** cluster_all ****
220309_ZnT3_P1, Method: dbscan, eps: 5.00 nm
Total tracks TID2: 2678, total cluster all: 2151
220309_Syp_P2, Method: dbscan, eps: 5.00 nm
Total tracks TID2: 2401, total cluster all: 1969

2022-09-22 11:32:26,335 [INFO] **** cluster_all_intersect ****
220309_ZnT3_P1 cluster with 220309_Syp_P2 cls 77/2151 = 0.04TID2 484/2678 = 0.18
220309_Syp_P2 cluster with 220309_ZnT3_P1 cls 77/1969 = 0.04TID2 407/2401 = 0.17

2022-09-22 11:32:30,927 [INFO] **** cluster_meas ****
220309_ZnT3_P1, Method dbscan, eps: 5.00 nm
Total tracks TID2: 2678, total clusters meas: 2299
220309_Syp_P2, Method dbscan, eps: 5.00 nm
Total tracks TID2: 2401, total clusters meas: 2106

2022-09-22 11:32:32,162 [INFO] **** cluster_a

2022-09-22 11:33:25,722 [INFO] **** cluster_meas ****
220309_ZnT3_P1, Method dbscan, eps: 40.00 nm
Total tracks TID2: 2678, total clusters meas: 195
220309_Syp_P2, Method dbscan, eps: 40.00 nm
Total tracks TID2: 2401, total clusters meas: 217

2022-09-22 11:33:28,998 [INFO] **** cluster_all ****
220309_ZnT3_P1, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 2678, total cluster all: 140
220309_Syp_P2, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 2401, total cluster all: 150

2022-09-22 11:33:29,229 [INFO] **** cluster_all_intersect ****
220309_ZnT3_P1 cluster with 220309_Syp_P2 cls 45/139 = 0.32TID2 823/2678 = 0.31
220309_Syp_P2 cluster with 220309_ZnT3_P1 cls 45/149 = 0.30TID2 616/2401 = 0.26

2022-09-22 11:33:34,909 [INFO] **** cluster_meas ****
220309_ZnT3_P1, Method dbscan, eps: 40.00 nm
Total tracks TID2: 2678, total clusters meas: 195
220309_Syp_P2, Method dbscan, eps: 40.00 nm
Total tracks TID2: 2401, total clusters meas: 217

2022-09-22 11:33:38,634 [INFO] **** cluster_all

In [16]:
eps_range = [2e-9, 5e-9, 1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 1e-7]
for label in npy_cwash:
    for afile in npy_cwash[label]:
        processFile(afile, eps_range=eps_range)

2022-09-22 11:34:38,097 [INFO] **** trim_min_localizations ****
220804_VGLUT1_P1 Removed 127/336 tracks with less than 3 localizations
220804_VGLUT1_P2 Removed 115/303 tracks with less than 3 localizations

2022-09-22 11:34:40,289 [INFO] **** cluster_tid ****
220804_VGLUT1_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.93 nm
Processed TID: 42 / 209, Total tracks TID2: 248
220804_VGLUT1_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.93 nm
Processed TID: 41 / 188, Total tracks TID2: 218

2022-09-22 11:34:40,291 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220804_VGLUT1_ROI01_First\220804_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-09
DBCLUSTER_EPS_ALL 2e-09
DBCLUSTER_EPS_MERGED_ALL 2e-09
DBCLUSTER_EPS_MERGED_MEAS 2e-09
ref_error {'se_xy': 0.0, 'se_z': 0.0, 'std_xy': 0.0, 'std_z': 0.0}
logger <Logger processlocalizations (INFO)>

2022-09-22 11:34:40,334 [INFO] 

2022-09-22 11:34:42,118 [INFO] **** cluster_all ****
220804_VGLUT1_P1, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 247, total cluster all: 97
220804_VGLUT1_P2, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 217, total cluster all: 129

2022-09-22 11:34:42,132 [INFO] **** cluster_all_intersect ****
220804_VGLUT1_P1 cluster with 220804_VGLUT1_P2 cls 27/96 = 0.28TID2 160/247 = 0.65
220804_VGLUT1_P2 cluster with 220804_VGLUT1_P1 cls 27/129 = 0.21TID2 107/217 = 0.49

export csv table
export paraview file
2022-09-22 11:34:42,374 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220804_VGLUT1_ROI01_First\220804_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 3e-08
DBCLUSTER_EPS_ALL 3e-08
DBCLUSTER_EPS_MERGED_ALL 3e-08
DBCLUSTER_EPS_MERGED_MEAS 3e-08
ref_error {'se_xy': 0.0, 'se_z': 0.0, 'std_xy': 0.0, 'std_z': 0.0}
logger <Logger processlocalizations (INFO)>

20

2022-09-22 11:34:44,536 [INFO] **** cluster_all ****
220804_VGLUT1_P1, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 247, total cluster all: 22
220804_VGLUT1_P2, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 217, total cluster all: 29

2022-09-22 11:34:44,551 [INFO] **** cluster_all_intersect ****
220804_VGLUT1_P1 cluster with 220804_VGLUT1_P2 cls 13/21 = 0.62TID2 235/247 = 0.95
220804_VGLUT1_P2 cluster with 220804_VGLUT1_P1 cls 13/29 = 0.45TID2 198/217 = 0.91

export csv table
export paraview file
2022-09-22 11:34:45,052 [INFO] **** trim_min_localizations ****
220811_VGLUT1_P1 Removed 927/2692 tracks with less than 3 localizations
220811_VGLUT1_P2 Removed 738/1813 tracks with less than 3 localizations

2022-09-22 11:35:45,443 [INFO] **** cluster_tid ****
220811_VGLUT1_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 11.11 nm
Processed TID: 354 / 1765, Total tracks TID2: 2023
220811_VGLUT1_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 11.11 nm
Processed TID: 225 / 1075, Total tracks TID2: 123

2022-09-22 11:36:06,134 [INFO] **** cluster_all ****
220811_VGLUT1_P1, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 2022, total cluster all: 644
220811_VGLUT1_P2, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 1235, total cluster all: 521

2022-09-22 11:36:06,265 [INFO] **** cluster_all_intersect ****
220811_VGLUT1_P1 cluster with 220811_VGLUT1_P2 cls 70/643 = 0.11TID2 1080/2022 = 0.53
220811_VGLUT1_P2 cluster with 220811_VGLUT1_P1 cls 70/520 = 0.13TID2 611/1235 = 0.49

2022-09-22 11:36:08,459 [INFO] **** cluster_meas ****
220811_VGLUT1_P1, Method dbscan, eps: 20.00 nm
Total tracks TID2: 2022, total clusters meas: 672
220811_VGLUT1_P2, Method dbscan, eps: 20.00 nm
Total tracks TID2: 1235, total clusters meas: 550

2022-09-22 11:36:09,374 [INFO] **** cluster_all ****
220811_VGLUT1_P1, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 2022, total cluster all: 644
220811_VGLUT1_P2, Method: dbscan, eps: 20.00 nm
Total tracks TID2: 1235, total cluster all: 521

2022-09-22 11:36:09,503 

2022-09-22 11:36:39,324 [INFO] **** cluster_meas ****
220811_VGLUT1_P1, Method dbscan, eps: 100.00 nm
Total tracks TID2: 2022, total clusters meas: 99
220811_VGLUT1_P2, Method dbscan, eps: 100.00 nm
Total tracks TID2: 1235, total clusters meas: 76

2022-09-22 11:36:41,572 [INFO] **** cluster_all ****
220811_VGLUT1_P1, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 2022, total cluster all: 70
220811_VGLUT1_P2, Method: dbscan, eps: 100.00 nm
Total tracks TID2: 1235, total cluster all: 42

2022-09-22 11:36:41,700 [INFO] **** cluster_all_intersect ****
220811_VGLUT1_P1 cluster with 220811_VGLUT1_P2 cls 14/69 = 0.20TID2 69/2022 = 0.03
220811_VGLUT1_P2 cluster with 220811_VGLUT1_P1 cls 14/41 = 0.34TID2 80/1235 = 0.06

2022-09-22 11:36:44,751 [INFO] **** cluster_meas ****
220811_VGLUT1_P1, Method dbscan, eps: 100.00 nm
Total tracks TID2: 2022, total clusters meas: 99
220811_VGLUT1_P2, Method dbscan, eps: 100.00 nm
Total tracks TID2: 1235, total clusters meas: 76

2022-09-22 11:36:46,908 [I

In [12]:
'%.0f' % (3e-8*1e9)

'30'